# Mumbai Indians Advanced Analytics & Visual Insights

## Objectives
This notebook provides deep, data-driven analysis for the Mumbai Indians (MI) franchise. As a prospective analyst for MI, this report focuses on:
1. **Match Progression Analysis**: How do MI build their innings over overs?
2. **Toss Impact**: Does winning the toss correlate with match wins?
3. **Dismissal Patterns**: How do top batsmen usually get out?
4. **Partnership Analysis**: Who are the most successful batting pairs?
5. **Venue Fortress**: Where is MI most dominant?
6. **Bowler Lethality**: Strike rates in different phases (Powerplay vs Death).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (14, 7)
plt.rcParams["font.size"] = 12
MI_COLOR = "#004BA0" # Mumbai Indians Blue

In [ ]:
# --- LOAD DATA ---
# Ensure previous steps created these CSVs
try:
    df_fact = pd.read_csv("MI_Fact_Deliveries.csv")
    df_matches = pd.read_csv("MI_Dim_Matches.csv")
    print("Data loaded successfully.")
except FileNotFoundError:
    print("❌ Error: Required CSV files not found. Please run the previous data preparation notebook first.")

## 1. Toss Impact Analysis
**Hypothesis**: Winning the toss is crucial for MI matches.

In [ ]:
# Simplified Toss Logic
# Create a boolean column directly in the dataframe
toss_analysis = df_matches.copy()

# Check if toss winner matches match winner
toss_analysis["toss_match_win_correlation"] = np.where(
    toss_analysis["toss_winner"] == toss_analysis["match_won_by"], 
    "Won Match", 
    "Lost Match"
)

# Filter for only when MI won the toss to see their conversion rate
# OR look at overall matches played by MI
mi_toss_wins = toss_analysis[toss_analysis["toss_winner"] == "Mumbai Indians"]

plt.figure(figsize=(8, 8))
explode = (0.05, 0)
mi_toss_wins["toss_match_win_correlation"].value_counts().plot(
    kind='pie', 
    autopct='%1.1f%%', 
    colors=['#66b3ff','#ff9999'], 
    explode=explode, 
    shadow=True, 
    startangle=90
)
plt.title("When MI Wins the Toss, Do They Win the Match?", fontsize=16, fontweight='bold')
plt.ylabel("")
plt.show()

## 2. Venue Fortress Analysis
**Insight**: Identifying venues where MI has the highest win percentage (min 5 matches).

In [ ]:
venue_wins = df_matches.groupby("venue").agg(
    total_matches=("match_id", "nunique"),
    wins=("match_won_by", lambda x: (x == "Mumbai Indians").sum())
).reset_index()

venue_wins["win_pct"] = (venue_wins["wins"] / venue_wins["total_matches"]) * 100
venue_wins = venue_wins[venue_wins["total_matches"] >= 5].sort_values("win_pct", ascending=False).head(10)

plt.figure(figsize=(12, 6))
sns.barplot(data=venue_wins, x="win_pct", y="venue", palette="Blues_r")
plt.xlabel("Win Percentage (%)")
plt.ylabel("Venue")
plt.title("MI's Fortress: Top Venues by Win % (Min 5 Matches)", fontsize=16, fontweight='bold')
plt.axvline(x=50, color='red', linestyle='--', alpha=0.5, label="50% Win Rate")
plt.legend()
plt.show()

## 3. Partnership Analysis (The Backbone)
**Insight**: Which batting pairs have scored the most runs together for MI?

In [ ]:
# Filter only MI batting data
mi_batting_only = df_fact[df_fact["batting_team"] == "Mumbai Indians"].copy()

# Create Partnership Column (Sorted tuple of batter & non-striker to handle A-B = B-A)
mi_batting_only["partners"] = mi_batting_only.apply(
    lambda x: " & ".join(sorted([str(x["batter"]), str(x["non_striker"])])), axis=1
)

partnerships = mi_batting_only.groupby("partners")["runs_total"].sum().reset_index()
top_partnerships = partnerships.sort_values("runs_total", ascending=False).head(10)

plt.figure(figsize=(12, 6))
sns.barplot(data=top_partnerships, x="runs_total", y="partners", palette="viridis")
plt.title("Top 10 Batting Partnerships for Mumbai Indians (All-Time)", fontsize=16, fontweight='bold')
plt.xlabel("Total Partnership Runs")
plt.ylabel("Batting Pair")
for index, row in top_partnerships.iterrows():
    plt.text(row.runs_total + 50, index, f"{row.runs_total}", va='center')
plt.show()

## 4. Dismissal Patterns of Top Batsmen
**Insight**: How does Rohit Sharma and Kieron Pollard usually get out? This reveals weaknesses.

In [ ]:
top_batsmen = ["RG Sharma", "KA Pollard", "SK Yadav"]
dismissals = mi_batting_only[
    (mi_batting_only["player_out"].isin(top_batsmen)) & 
    (mi_batting_only["wicket_kind"].notna())
]

plt.figure(figsize=(14, 6))
sns.countplot(data=dismissals, x="player_out", hue="wicket_kind", palette="deep")
plt.title("Dismissal Modes of Top MI Batters", fontsize=16, fontweight='bold')
plt.xlabel("Batsman")
plt.ylabel("Count of Dismissals")
plt.legend(title="Dismissal Type", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

## 5. Death Over Specialist Analysis (Bowlers)
**Insight**: Who performs best in overs 16-20? (Low Economy + Low Strike Rate)

In [ ]:
mi_bowling_only = df_fact[df_fact["bowling_team"] == "Mumbai Indians"].copy()

# Define Death Overs (Overs 16-20). In Data, Overs are typically 0-19 or 1-20.
# Assuming 0-19: Death is > 15. Assuming 1-20: Death is > 15. Filter works for both >15.
death_overs = mi_bowling_only[mi_bowling_only["over"] > 15]

death_stats = death_overs.groupby("bowler").agg(
    balls=("valid_ball", "sum"),
    runs=("runs_total", "sum"),
    wickets=("bowler_wicket", "sum")
).reset_index()

# Filter for significant bowlers
# LOWERING THRESHOLD to 48 balls (approx 8 overs in death) to ensure data shows up
threshold_balls = 48 
death_stats_filtered = death_stats[death_stats["balls"] > threshold_balls].copy()

print(f"Found {len(death_stats_filtered)} bowlers with > {threshold_balls} balls in Death overs.")

if not death_stats_filtered.empty:
    death_stats_filtered["economy"] = death_stats_filtered["runs"] / (death_stats_filtered["balls"]/6)
    # prevent division by zero for strike rate
    death_stats_filtered["strike_rate"] = death_stats_filtered.apply(
        lambda x: x["balls"] / x["wickets"] if x["wickets"] > 0 else 60, axis=1
    )

    plt.figure(figsize=(12, 8))
    sns.scatterplot(
        data=death_stats_filtered, 
        x="economy", 
        y="strike_rate", 
        size="wickets", 
        sizes=(100, 1000), 
        hue="economy", 
        palette="RdYlGn_r"
    )

    for line in range(0, death_stats_filtered.shape[0]):
        # Simple collision avoidance (very basic)
        y_pos = death_stats_filtered.strike_rate.iloc[line]
        
        plt.text(
            death_stats_filtered.economy.iloc[line]+0.05, 
            y_pos, 
            death_stats_filtered.bowler.iloc[line], 
            horizontalalignment='left', 
            size='medium', 
            weight='bold'
        )

    plt.title("Death Over Specialists (Overs 16-20)", fontsize=16, fontweight='bold')
    plt.xlabel("Economy Rate (Lower is Better)")
    plt.ylabel("Strike Rate (Balls per Wicket - Lower is Better)")
    plt.show()
else:
    print("No bowlers found meeting the criteria.")

## 6. Run Accumulation Heatmap (Over vs Season)
**Insight**: Visualizing scoring intensity across overs for each season.

In [ ]:
# Aggregating runs by Season and Over
heatmap_data = mi_batting_only.groupby(["season", "over"])["runs_total"].sum().reset_index()
heatmap_pivot = heatmap_data.pivot(index="season", columns="over", values="runs_total")

plt.figure(figsize=(15, 8))
sns.heatmap(heatmap_pivot, cmap="YlOrRd", linewidths=0.5)
plt.title("Heatmap of Runs Scored by Over Across Seasons", fontsize=16, fontweight='bold')
plt.xlabel("Over Number (1-20)")
plt.ylabel("Season")
plt.show()